In [9]:
import numpy as np

import torch
import torch.nn as nn

In [10]:
class MyDataParallel(nn.DataParallel):
    def __init__(self, model):
        super().__init__(model)

    def __getattr__(self, name):
        try:
            return super(MyDataParallel, self).__getattr__(name)
        except AttributeError:
            return getattr(self.module, name)

    def __setattr__(self, name, value):
        try:
            return super(MyDataParallel, self).__setattr__(name, value)
        except AttributeError:
            return setattr(self.module, name, value)
        
class ReLULinearDeepLiftModel(nn.Module):
    r"""
        Simple architecture similar to:
        https://github.com/marcoancona/DeepExplain/blob/master/deepexplain/tests/test_tensorflow.py#L65
    """

    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(3, 1, bias=False)
        self.l2 = nn.Linear(3, 1, bias=False)
        self.l1.weight = nn.Parameter(torch.tensor([[3.0, 1.0, 0.0]]))
        self.l2.weight = nn.Parameter(torch.tensor([[2.0, 3.0, 0.0]]))
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(2, 1, bias=False)
        self.l3.weight = nn.Parameter(torch.tensor([[1.0, 1.0]]))

    def forward(self, x1, x2):
        return self.l3(self.relu(torch.cat([self.l1(x1), self.l2(x2)], axis=1)))


In [ ]:
dp_model = MyDataParallel(ReLULinearDeepLiftModel()).cuda()

model = ReLULinearDeepLiftModel()

In [ ]:
def pre_hook(module, baseline_inputs):
    inputs = baseline_inputs[0]
    baselines = baseline_inputs[1]
    res = tuple(torch.cat([input, baseline]) for input, baseline in zip(inputs, baselines))

    print('res: ', res)

    return res
  
dp_model.module.register_forward_pre_hook(pre_hook)

model.register_forward_pre_hook(pre_hook)

In [ ]:
'''
def _forward_hook(module, inputs, outputs):
    print('inputs: ', inputs)

dp_model.module.relu.register_forward_hook(_forward_hook)
'''

In [ ]:
inp1 = torch.tensor([[-10.0, 1.0, -5.0], [-23.0, 22.0, -12.0], [-10.0, 1.0, -5.0]], requires_grad=True).cuda()
inp2 = torch.tensor([[3.0, 3.0, 1.0], [13.0, 23.0, 31.0], [3.0, 3.0, 1.0]], requires_grad=True).cuda()

base1 = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], requires_grad=True).cuda()
base2 = torch.tensor([[1.0, 1.0, 0.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]], requires_grad=True).cuda()

inputs = (inp1, inp2)
baselines = (base1, base2)
pred = dp_model(*inputs)

print('pred: ', pred)

# ----- 

inp1 = torch.tensor([[-10.0, 1.0, -5.0], [-23.0, 22.0, -12.0], [-10.0, 1.0, -5.0]], requires_grad=True)
inp2 = torch.tensor([[3.0, 3.0, 1.0], [13.0, 23.0, 31.0], [3.0, 3.0, 1.0]], requires_grad=True)

base1 = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], requires_grad=True)
base2 = torch.tensor([[1.0, 1.0, 0.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0]], requires_grad=True)

inputs = (inp1, inp2)

baselines = (base1, base2)
pred = model(*inputs)
print('pred: ', pred)

In [ ]:
t1 = torch.rand(1,2,3)
t2 = torch.rand(1,2,3)

In [ ]:
t = torch.cat([t1, t2], axis=1)

In [ ]:
print(t1.shape)
print(t2.shape)
print(t.shape)

In [ ]:
a = torch.randn(50, 80) #tensor of size 50 x 80
b = torch.split(a, 40, dim=1) # it returns a tuple

spl = t.split(2, dim=1)

In [ ]:
def _forward_hook(module, inputs, outputs):
    if type(module) == nn.ReLU:
        print(inputs)
        print(outputs)
    module.a = nn.Parameter(torch.zeros(1,2))
    
def _forward_hook_on_relu(module):
        module.register_forward_hook(_forward_hook)
        


In [11]:
model = ReLULinearDeepLiftModel()

inp1 = torch.tensor([[-10.,   1.,  -5.], [ -0.,   0.,  -0.]], requires_grad=True, device=torch.device('cuda:0'))
inp2 = torch.tensor([[3., 3., 1.], [0., 0., 0.]], requires_grad=True)

#tensor([[-10.,   1.,  -5.],
#        [ -0.,   0.,  -0.]], device='cuda:0', grad_fn=<CatBackward>), tensor([[3., 3., 1.],
#        [0., 0., 0.]], device='cuda:0', grad_fn=<CatBackward>)
inputs = (inp1, inp2)


model.apply(_forward_hook_on_relu)
model(*inputs)

NameError: name '_forward_hook_on_relu' is not defined

In [3]:
from captum.attr import DeepLift
net = ReLULinearDeepLiftModel().cuda()
inp1 = torch.tensor([[-10.0, 1.0, -5.0], [1.9, 2.0, 1.9],[1.1, 1.2, 1.3], [1.9, 2.0, 1.9]], requires_grad=True).cuda()
inp2 = torch.tensor([[3.0, 3.0, 1.0], [1.2, 3.0, 2.3],[1.1, 1.2, 1.3], [1.2, 3.0, 2.3]], requires_grad=True).cuda()
dl = DeepLift(net)
dl.attribute((inp1, inp2))

forward_func:  ReLULinearDeepLiftModel(
  (l1): Linear(in_features=3, out_features=1, bias=False)
  (l2): Linear(in_features=3, out_features=1, bias=False)
  (relu): ReLU()
  (l3): Linear(in_features=2, out_features=1, bias=False)
)


/home/narine/captum/captum/attr/_core/deep_lift.py:239: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  after the attribution is finished"""


(tensor([[0.0000, -0.0000, 0.0000],
         [5.7000, 2.0000, 0.0000],
         [3.3000, 1.2000, 0.0000],
         [5.7000, 2.0000, 0.0000]], device='cuda:0', grad_fn=<MulBackward0>),
 tensor([[6.0000, 9.0000, 0.0000],
         [2.4000, 9.0000, 0.0000],
         [2.2000, 3.6000, 0.0000],
         [2.4000, 9.0000, 0.0000]], device='cuda:0', grad_fn=<MulBackward0>))

In [4]:
from captum.attr import DeepLift, LayerDeepLift
net = nn.DataParallel(ReLULinearDeepLiftModel().cuda())
inp1 = torch.tensor([[-10.0, 1.0, -5.0], [1.9, 2.0, 1.9], [1.1, 1.2, 1.3], [1.9, 2.0, 1.9]], requires_grad=True).cuda()
inp2 = torch.tensor([[3.0, 3.0, 1.0], [1.2, 3.0, 2.3], [1.1, 1.2, 1.3], [1.2, 3.0, 2.3]], requires_grad=True).cuda()
dl = DeepLift(net)
dl.attribute((inp1, inp2))

forward_func:  DataParallel(
  (module): ReLULinearDeepLiftModel(
    (l1): Linear(in_features=3, out_features=1, bias=False)
    (l2): Linear(in_features=3, out_features=1, bias=False)
    (relu): ReLU()
    (l3): Linear(in_features=2, out_features=1, bias=False)
  )
)
[0, 1, 2, 3, 4, 5, 6, 7]


(tensor([[0.0000, -0.0000, 0.0000],
         [5.7000, 2.0000, 0.0000],
         [3.3000, 1.2000, 0.0000],
         [5.7000, 2.0000, 0.0000]], device='cuda:0', grad_fn=<MulBackward0>),
 tensor([[6.0000, 9.0000, 0.0000],
         [2.4000, 9.0000, 0.0000],
         [2.2000, 3.6000, 0.0000],
         [2.4000, 9.0000, 0.0000]], device='cuda:0', grad_fn=<MulBackward0>))

In [5]:
from captum.attr import DeepLift, LayerDeepLift
net = nn.DataParallel(ReLULinearDeepLiftModel().cuda())
inp1 = torch.tensor([[-10.0, 1.0, -5.0], [1.9, 2.0, 1.9], [1.1, 1.2, 1.3], [1.9, 2.0, 1.9]], requires_grad=True).cuda()
inp2 = torch.tensor([[3.0, 3.0, 1.0], [1.2, 3.0, 2.3], [1.1, 1.2, 1.3], [1.2, 3.0, 2.3]], requires_grad=True).cuda()
dl = LayerDeepLift(net, net.module.l2)
dl.attribute((inp1, inp2))

forward_func:  DataParallel(
  (module): ReLULinearDeepLiftModel(
    (l1): Linear(in_features=3, out_features=1, bias=False)
    (l2): Linear(in_features=3, out_features=1, bias=False)
    (relu): ReLU()
    (l3): Linear(in_features=2, out_features=1, bias=False)
  )
)
[0, 1, 2, 3, 4, 5, 6, 7]


tensor([[15.0000],
        [11.4000],
        [ 5.8000],
        [11.4000]], device='cuda:0', grad_fn=<MulBackward0>)

In [6]:
from captum.attr import DeepLift, LayerDeepLift
net = ReLULinearDeepLiftModel().cuda()
inp1 = torch.tensor([[-10.0, 1.0, -5.0], [1.9, 2.0, 1.9], [1.1, 1.2, 1.3], [1.9, 2.0, 1.9]], requires_grad=True).cuda()
inp2 = torch.tensor([[3.0, 3.0, 1.0], [1.2, 3.0, 2.3], [1.1, 1.2, 1.3], [1.2, 3.0, 2.3]], requires_grad=True).cuda()
dl = LayerDeepLift(net, net.l2)
dl.attribute((inp1, inp2))

forward_func:  ReLULinearDeepLiftModel(
  (l1): Linear(in_features=3, out_features=1, bias=False)
  (l2): Linear(in_features=3, out_features=1, bias=False)
  (relu): ReLU()
  (l3): Linear(in_features=2, out_features=1, bias=False)
)


tensor([[15.0000],
        [11.4000],
        [ 5.8000],
        [11.4000]], device='cuda:0', grad_fn=<MulBackward0>)

In [7]:
k.view(k.shape[1:], 2)[:,0]

NameError: name 'k' is not defined

In [8]:
t = torch.tensor([[1],[2],[3],[4],[5],[6]])
t.shape

torch.Size([6, 1])

In [ ]:
t.view((3,)+ (2,) + (1,))

In [18]:
inp1

tensor([[-10.0000,   1.0000,  -5.0000],
        [  1.9000,   2.0000,   1.9000],
        [  1.1000,   1.2000,   1.3000],
        [  1.9000,   2.0000,   1.9000]], device='cuda:0',
       grad_fn=<CopyBackwards>)

In [19]:
fn = lambda out: out.chunk(2)[0]

In [20]:
inp1[0: int(len(inp1) / 2)]

tensor([[-10.0000,   1.0000,  -5.0000],
        [  1.9000,   2.0000,   1.9000]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [26]:
torch.cat((inp1,inp2))

tensor([[-10.0000,   1.0000,  -5.0000],
        [  1.9000,   2.0000,   1.9000],
        [  1.1000,   1.2000,   1.3000],
        [  1.9000,   2.0000,   1.9000],
        [  3.0000,   3.0000,   1.0000],
        [  1.2000,   3.0000,   2.3000],
        [  1.1000,   1.2000,   1.3000],
        [  1.2000,   3.0000,   2.3000]], device='cuda:0',
       grad_fn=<CatBackward>)

In [15]:
inp = torch.tensor(0)
inp.size(0)

IndexError: dimension specified as 0 but tensor has no dimensions